pip installs

In [32]:
#pip install requests

imports

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.model_selection import train_test_split 
# from sklearn import datasets, tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, recall_score, precision_score
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio  # me permite dibujar en un notebook
pio.renderers.default = "notebook"  # me permite dibujar en un notebook
import plotly.express as px
import plotly.figure_factory as ff
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
import requests
import os

load/write labels and parameters

In [299]:

data_start_label='Parameter'
Parameter_data_rows = ['Unit','HighL','LowL','Tests#','Patterns']
HardBinDictName='HardBinName'
SoftBinDictName='SoftBinName'
header_rows_drop = 4
datatypes_per_unit = {
    'VOLTS':'float64',
    'Ohms':'float64',
    'V':'float64',
    'uSECONDS':'float64',
    'mAMPS':'float64',
    ' ':'float64',
    'code':'int64',
    'C':'float64',
    'LSB':'float64',
    'deg':'float64',
    'PF':'float64',
    'KHERTZ':'float64',
    'MHERTZ':'float64',
    'KkHz':'float64',
    '%':'float64',
    'uAMPS':'float64',
    'nnA':'float64',
    'nAMPS':'float64'}
testdatacols=[' SBIN','HBIN','DIE_X','DIE_Y','SITE','TIME','TOTAL_TESTS','LOT_ID','WAFER_ID']
confdict={'data_start_label':data_start_label,'Parameter_data_rows':Parameter_data_rows,'testdatacols':testdatacols,'HardBinDictName':HardBinDictName,'SoftBinDictName':SoftBinDictName,'datatypes_per_unit':datatypes_per_unit}



Load csv
generate df with data
generate HardBinDict
generate SoftBinDict
generate general_data

In [305]:
def load_data (_file_location, _confdict):
    _rawdf = pd.read_csv(_file_location)  # carga csv
    #agregar busqueda de "Parameter" y generacion de exception en caso de no hallarlo así no se pierde en caso de no haber
    _data_start_col_value=(_rawdf.iloc[:,0]==_confdict.get('data_start_label')).idxmax()+1  # search for label in table data start
    _df = pd.read_csv(_file_location, header=_data_start_col_value, index_col=_confdict.get('data_start_label'))  # load table data from csv into df
    _df.drop(columns=_df.columns[-1], axis=1, inplace=True) #extra column without data

    #separate testdata and parameter data (might get input to make this optional)
    _dftestdata = _df[_confdict.get('testdatacols')]
    _df.drop(columns=_confdict.get('testdatacols'), inplace=True)
    _dfpardata=_df.loc[_confdict.get('Parameter_data_rows')]
    _df.drop(_confdict.get('Parameter_data_rows'), inplace=True)
    _df.reset_index(drop=True)
    #_dfpardata.loc['Unit'].replace(confdict.get('datatypes_per_unit'))

    # 
    _rawdf = _rawdf.head(_data_start_col_value-1)  # drop table data
    _rawdf = _rawdf.tail(-header_rows_drop)  # drop 4 rows from the header
    _HardBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('HardBinDictName')].iloc[:,[1,2]])
    _HardBinDict = _HardBinDict.set_index(_HardBinDict.iloc[:, 0]).iloc[:,1]
    _HardBinDict = _HardBinDict.to_dict()
    _SoftBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('SoftBinDictName')].iloc[:,[1,2]])
    _SoftBinDict = _SoftBinDict.set_index(_SoftBinDict.iloc[:, 0]).iloc[:,1]
    _SoftBinDict = _SoftBinDict.to_dict()
    _rawdf = _rawdf[(_rawdf.iloc[:,0]!=_confdict.get('HardBinDictName'))&(_rawdf.iloc[:,0]!=_confdict.get('SoftBinDictName'))]  # drop softbin and hardbin
    _rawdf.dropna(how='all')
    _general_data = _rawdf.iloc[:,[0,1]]  # global data extracted by position
    _general_data = _general_data.set_index(_general_data.iloc[:, 0]).iloc[:,1]
    _general_data = _general_data.to_dict()

    # input corrections
    _dfpardata['HSR_SMM ANGLE_D1']['Unit'] = 'deg'  # aparece la unidad como "code" y claramente es "deg"
    _dfpardata['HSR_SMM ANGLE_D2']['Unit'] = 'deg'  # aparece la unidad como "code" y claramente es "deg"
    _df = _df.replace(r'^\s*$', '99999999', regex=True)  # cuando un chip vuela y no se le hacen más ensayos, se le pone un " " en vez del resultado np.NaN

    _coldtypes=_dfpardata.loc['Unit'].replace(_confdict.get('datatypes_per_unit')).fillna('float64').to_dict()  # por defecto float por haber tantos sin unidad no enteros
    _df = _df.astype(_coldtypes)

    return (_df, _general_data,_dfpardata,_dftestdata, _HardBinDict, _SoftBinDict)

In [307]:
file_location1 = "C:/Users/ggrinberg/Downloads/QR6129X1.csv"
file_location2 = "C:/Users/ggrinberg/Downloads/QH6129X2.csv"
file_location3 = "C:/Users/ggrinberg/Downloads/QR6129X3.csv"
file_location4 = "C:/Users/ggrinberg/Downloads/QH6129X4.csv"
(df1, general_data1 ,dfpardata1, dftestdata1, HardBinDict, SoftBinDict) = load_data (file_location1, confdict)
(df2, general_data2 ,dfpardata2, dftestdata2, HardBinDict, SoftBinDict) = load_data (file_location2, confdict)
(df3, general_data3 ,dfpardata3, dftestdata3, HardBinDict, SoftBinDict) = load_data (file_location3, confdict)
(df4, general_data4 ,dfpardata4, dftestdata4, HardBinDict, SoftBinDict) = load_data (file_location4, confdict)

In [282]:
df3['SENT Trigger Threshold L Code 0_D1']['PID-120']

nan

In [303]:
df1

,VCC1 OS Pre_D1,VCC1 OS Pre_D2,PWM OS Pre_D1,PWM OS Pre_D2,CSN OS Pre_D1,CSN OS Pre_D2,SCLK OS Pre_D1,SCLK OS Pre_D2,MOSI OS Pre_D1,MOSI OS Pre_D2,...,MOSI OS Pre_D1.1,MOSI OS Pre_D2.1,MISO OS Pre_D1.1,MISO OS Pre_D2.1,AU OS Pre_D1.1,AU OS Pre_D2.1,BV OS Pre_D1.1,BV OS Pre_D2.1,IW OS Pre_D1.1,IW OS Pre_D2.1
Parameter,,,,,,,,,,,,,,,,,,,,,
PID-1,-18.53508,-19.82092,-0.553776,-0.555167,-0.525353,-0.533075,-0.524304,-0.533188,-0.525596,-0.533693,...,-0.512110,-0.519495,-0.511586,-0.519944,-0.508364,-0.516348,-0.509563,-0.518352,-0.509731,-0.518746
PID-2,-18.55505,-19.79125,-0.559074,-0.559978,-0.527975,-0.535323,-0.527582,-0.536147,-0.528069,-0.535791,...,-0.513871,-0.521199,-0.514227,-0.521181,-0.510368,-0.517434,-0.511960,-0.519589,-0.512541,-0.520619
PID-3,-18.55004,-19.68912,-0.559133,-0.559822,-0.529679,-0.526557,-0.530129,-0.526332,-0.529960,-0.527044,...,-0.516100,-0.512545,-0.516399,-0.512377,-0.513084,-0.510092,-0.514620,-0.511721,-0.515425,-0.512227
PID-4,-18.62146,-19.78100,-0.559094,-0.560114,-0.527975,-0.532757,-0.528518,-0.533225,-0.529080,-0.533356,...,-0.514170,-0.518839,-0.514358,-0.519326,-0.511024,-0.515449,-0.513253,-0.516985,-0.513028,-0.518277
PID-5,-18.54693,-19.78290,-0.558899,-0.560290,-0.526214,-0.534218,-0.526308,-0.534087,-0.526982,-0.534218,...,-0.512204,-0.519326,-0.512204,-0.519626,-0.509675,-0.516048,-0.509956,-0.517734,-0.510780,-0.518877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID-141,-18.62479,-19.77288,-0.559698,-0.560465,-0.533819,-0.535080,-0.534137,-0.535323,-0.534418,-0.534855,...,-0.519640,-0.519663,-0.519228,-0.519720,-0.515313,-0.516123,-0.517823,-0.518483,-0.518872,-0.520019
PID-142,-18.62823,-19.79903,-0.559405,-0.559764,-0.533088,-0.535211,-0.533126,-0.535154,-0.533350,-0.534742,...,-0.518516,-0.520001,-0.519452,-0.519832,-0.515201,-0.517210,-0.517430,-0.518652,-0.518273,-0.519439
PID-143,-18.62625,-19.80957,-0.559503,-0.560368,-0.532807,-0.534592,-0.534137,-0.535229,-0.533800,-0.535211,...,-0.518947,-0.520338,-0.519209,-0.520619,-0.515444,-0.516573,-0.517711,-0.519101,-0.518535,-0.520207


In [298]:
datatypes_per_unit

{'VOLTS': 'float64',
 'Ohms': 'float64',
 'V': 'float64',
 'uSECONDS': 'float64',
 'mAMPS': 'float64',
 ' ': 'float64',
 'code': 'int64',
 'C': 'float64',
 'LSB': 'float64',
 'deg': 'float64',
 'PF': 'float64',
 'KHERTZ': 'float64',
 'MHERTZ': 'float64',
 'KkHz': 'float64',
 '%': 'float64',
 'uAMPS': 'float64',
 'nna': 'float64',
 'nAMPS': 'float64'}

In [124]:
dfpardata1.equals(dfpardata4)  # all min and max values are equal in all tests

True

In [167]:
df1.ge(dfpardata1.loc['LowL'], axis=1)

C:\Users\ggrinberg\AppData\Roaming\Python\Python311\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning:

invalid value encountered in greater_equal



,VCC1 OS Pre_D1,VCC1 OS Pre_D2,PWM OS Pre_D1,PWM OS Pre_D2,CSN OS Pre_D1,CSN OS Pre_D2,SCLK OS Pre_D1,SCLK OS Pre_D2,MOSI OS Pre_D1,MOSI OS Pre_D2,...,MOSI OS Pre_D1.1,MOSI OS Pre_D2.1,MISO OS Pre_D1.1,MISO OS Pre_D2.1,AU OS Pre_D1.1,AU OS Pre_D2.1,BV OS Pre_D1.1,BV OS Pre_D2.1,IW OS Pre_D1.1,IW OS Pre_D2.1
Parameter,,,,,,,,,,,,,,,,,,,,,
PID-1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID-141,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-142,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PID-143,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [304]:
faulted_by_param = (df1.ge(dfpardata1.loc['LowL'], axis=1) | df1.le(dfpardata1.loc['HighL'], axis=1))
#faulted_by_param.iloc[:,range(15,20)]

C:\Users\ggrinberg\AppData\Roaming\Python\Python311\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning:

invalid value encountered in greater_equal



TypeError: '>=' not supported between instances of 'float' and 'str'

In [260]:
df1.dtypes

VCC1 OS Pre_D1    float64
VCC1 OS Pre_D2    float64
PWM OS Pre_D1     float64
PWM OS Pre_D2     float64
CSN OS Pre_D1     float64
                   ...   
AU OS Pre_D2.1    float64
BV OS Pre_D1.1    float64
BV OS Pre_D2.1    float64
IW OS Pre_D1.1    float64
IW OS Pre_D2.1    float64
Length: 812, dtype: object

In [259]:
#coldtypes=dfpardata1.loc['Unit'].replace(datatypes_per_unit).to_dict()
coldtypes=dfpardata1.loc['Unit'].replace(datatypes_per_unit).fillna('int64').to_dict()
df1 = df1.astype(coldtypes)